# Pet-i Behavior Classification
반려견 활동 상태 예측 모델

## 1. 환경 설정
필요한 라이브러리를 임포트합니다.

In [ ]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# 노트북을 재현 가능한 상태로 설정
os.environ['PYTHONHASHSEED'] = '0'


## 2. 데이터 로드 및 전처리
`load_dataset` 함수를 정의하여 CSV 파일을 로드하고 전처리합니다.

In [ ]:
def load_dataset(data_dir='./dataset'):
    """
    센서 데이터 CSV 파일을 로드하고 특징(X)과 레이블(y)로 반환합니다.
    """
    try:
        move = pd.read_csv(os.path.join(data_dir, 'DogMoveData.csv'))
    except FileNotFoundError:
        raise FileNotFoundError(
            '데이터 파일을 찾을 수 없습니다. dataset 폴더에 DogMoveData.csv를 놓으세요.'
        )
    # 센서 관련 컬럼 선택
    sensor_cols = [
        'ABack_x','ABack_y','ABack_z','ANeck_x','ANeck_y','ANeck_z',
        'GBack_x','GBack_y','GBack_z','GNeck_x','GNeck_y','GNeck_z'
    ]
    X = move[sensor_cols].values
    y = move['Behavior_1'].values
    return X, y

# 데이터 로드 및 분할 예제
X, y_raw = load_dataset()
encoder = LabelEncoder()
y = encoder.fit_transform(y_raw)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)
# 스케일링
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
print(f'Train samples: {X_train.shape}, Test samples: {X_test.shape}')

## 3. 모델 정의
완전연결 신경망(MLP) 모델을 생성하는 `build_model` 함수를 정의합니다.

In [ ]:
def build_model(input_shape, num_classes):
    """
    입력 형태와 클래스 수에 따라 MLP 모델을 반환합니다.
    """
    model = keras.Sequential([
        layers.Dense(64, activation='relu', input_shape=input_shape),
        layers.Dense(32, activation='relu'),
        layers.Dense(num_classes, activation='softmax')
    ])
    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

# 모델 생성
model = build_model(input_shape=(X_train.shape[1],), num_classes=len(encoder.classes_))
model.summary()

## 4. 훈련 및 평가
훈련 함수 `train_model`을 정의하고 훈련 및 평가를 수행합니다.

In [ ]:
def train_model(model, X_train, y_train, X_test, y_test, epochs=35, batch_size=32):
    """
    모델을 훈련시키고 history를 반환합니다.
    """
    history = model.fit(
        X_train, y_train,
        validation_data=(X_test, y_test),
        epochs=epochs,
        batch_size=batch_size,
        verbose=2
    )
    test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)
    print(f'테스트 정확도: {test_acc:.4f}, 테스트 손실: {test_loss:.4f}')
    return history

# 모델 훈련 및 평가
history = train_model(model, X_train, y_train, X_test, y_test)

## 5. 예측 및 결과 확인
샘플 데이터를 예측하고 클래스 이름으로 변환합니다.

In [ ]:
sample_idx = 0
sample = X_test[sample_idx:sample_idx+1]
pred = model.predict(sample)
pred_class = encoder.inverse_transform([np.argmax(pred)])[0]
print(f'샘플 {sample_idx} 예측 클래스: {pred_class}')

## 6. 시각화
훈련 과정의 정확도 및 손실 곡선을 표시합니다.

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['Train', 'Validation'])
plt.show()

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['Train', 'Validation'])
plt.show()